In [1]:
# =======================================================
# Notebook setup (run before the rest of the notebook)
# =======================================================

%load_ext autoreload
%autoreload 2
%matplotlib widget

from sklearn.neighbors import KernelDensity
from util import nab
import numpy as np
from matplotlib import pyplot as plt

data_folder = '../data/nab'
file_name = 'realKnownCause/nyc_taxi.csv'
data, labels, windows = nab.load_series(file_name, data_folder)

# Anomaly Detection and Kernel Density Estimation

## Problem Formalization

**How do we formalize this problem?**

A possible approach: we know that anomalies are (often) unlikely

* If we can estimate the probability of every occurring observation $x$
* ...Then we can spot anomalies based on their _low probability_


> **We turn a liability into a strenght!**

## Problem Formalization

**We can check our intuition on our data**

This is (roughly) the distribution _over all the data_

In [2]:
vmax = data['value'].max()
nab.plot_histogram(data['value'], vmax=vmax, bins=20)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Problem Formalization

**We can check our intuition on our data**

This is (roughly) the distribution _around the first anomaly_:

In [3]:
w0_start, w0_end = windows.loc[0]['begin'], windows.loc[0]['end']
data_anomaly0 = data[(data.index >= w0_start) & (data.index < w0_end)]
nab.plot_histogram(data_anomaly0['value'], vmax=vmax, bins=30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* There seems to be a significant difference


## Problem Formalization

**How do we formalize this problem?**

A possible approach: we know that anomalies are (often) unlikely

* If we can estimate the probability of every occurring observation $x$
* ...Then we can spot anomalies based on their _low probability_


> **We turn a liability into a strength!**

**Formally, our detection condition can be stated as:**

$$
f(x) \leq \theta
$$

* Where $f(x)$ is a _Probability Density Function (PDF)_
* ...And $\theta$ is a (scalar) threshold

**What do we need to make this work?**

## Density Estimation

**We need one way to _estimate probability densities_**

For some random process with n-dimensional variable $x$:

* Given the true density function $f^*(x): \mathbb{R}^n \rightarrow \mathbb{R}^+$
* ...And a second function $f(x, \omega)$ with the same input, and parameters $\omega$

We want to _make the two as similar as possible_

**What about modeling that as supervised learning?**

Given some suitable loss function $L({\bf y}, {\bf y^*})$, that would lead to:

$$
\text{argmin}_{\omega} L(f(\hat{\bf x}, \omega), f^*(\hat{\bf x}))
$$

* where $\hat{\bf x}$ represents the training data

## Density Estimation

**Unfortunately, this approach _cannot work_**

...Because typically _we do not have access_ to the true density $f^*$

> **Density estimation is an _unsupervised_ learning problem**

**It can be solved via a number of techniques:**

* Simple histograms
* Kernel Density Estimation
* Gaussian Mixture Models
* [Normalizing Flows](https://arxiv.org/abs/1505.05770)
* [Non Volume Preserving (NVP) transformations](https://arxiv.org/abs/1605.08803)

**Which one shall we pick?**

## Our Friend, Occam's Razor

**We will go with Occam's razor**

It's a philosophical principle stating that:

> **Between two hypotheses, the simpler one is usually correct**

<center><img src="assets/dt980102dhc0.gif" width="800px"/></center>

## Our Friend, Occam's Razor

**We will go with Occam's razor**

It's a philosophical principle stating that:

> **Between two hypotheses, the simpler one is usually correct**

**For its simplicity we will pick _Kernel Density Estimation_**

* This will be returning principle in the course
* ...We will typically try _simpler approaches first_
* Especially at the beginning! Brace up for a slow start

# Kernel Density Estimation

## Kernal Density Estimation

**In _Kernel Density Estimation (KDE)_, the main idea is that:**

* Wherever (in input space) there is a sample
* ...It's likely that there are more

So, we assume that _each training sample is the center for a density "kernel"_

**Formally, the kernel $K(x, h)$ is just a valid PDF:**

* $x$ is the input variable (scalar or vector)
* $h$ is a parameter (resp. scalar or matrix) called _bandwidth_

Typical kernels: Gaussian, exponential, cosine, linear...

## Kernels

An example with one sample and a _Guassian_ kernel:

In [4]:
x = np.array(0.5).reshape(1,1) # single sample
kde = KernelDensity(kernel='gaussian', bandwidth=0.1) # build the estimator
kde.fit(x) # fit the estimator on the data
# We use a plotting function from our module
nab.plot_density_estimator_1D(kde, xr=np.linspace(0, 1, 200))
ymin, ymax = plt.ylim()
plt.vlines(x, ymin, ymax, color='tab:red')
plt.ylim((ymin, ymax)); # ; = suppress output

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kernel

An example with one sample and a _Tophat_ kernel:

In [5]:
x = np.array(0.5).reshape(1,1) # single sample
kde = KernelDensity(kernel='tophat', bandwidth=0.1) # build the estimator
kde.fit(x) # fit the estimator on the data
# We use a plotting function from our module
nab.plot_density_estimator_1D(kde, xr=np.linspace(0, 1, 200))
ymin, ymax = plt.ylim()
plt.vlines(x, ymin, ymax, color='tab:red')
plt.ylim((ymin, ymax)); # ; = suppress output

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kernel

An example with one sample and a _linear_ kernel:

In [6]:
x = np.array(0.5).reshape(1,1) # single sample
kde = KernelDensity(kernel='linear', bandwidth=0.1) # build the estimator
kde.fit(x) # fit the estimator on the data
# We use a plotting function from our module
nab.plot_density_estimator_1D(kde, xr=np.linspace(0, 1, 200))
ymin, ymax = plt.ylim()
plt.vlines(x, ymin, ymax, color='tab:red')
plt.ylim((ymin, ymax)); # ; = suppress output

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kernels

**As an example, a _Gaussian kernel_ in sklearn is given by:**

$$
K(x, h) \propto e^{-\frac{x^2}{2h^2}}
$$

* The $\propto$ ("proportional to") means that there is an _implicit normalization constant_
* The constant is handled by scikit-learn (in an efficient way)

**All kernels in KDE:**

* Are by default _zero-centered_ (their mode is at 0)
* Can be relocated via an affine transformation (i.e. summing a constant to $x$)

**In practice:**

$$
K(x - \mu, h)
$$

...Gives the value at $x$ of a kernel centered on $\mu$


## Kernel Density Estimation

**The estimated density of any point is obtained as a _kernel average_:**

$$
f(x, \hat{\bf x}, h) = \frac{1}{m} \sum_{i=0}^m K(x - \hat{x}_i, h)
$$

* $x$ is the input for which we want an estimate
* $\hat{\bf x}$ is the matrix with the training samples
  - The training samples are _part of the model parameters_
* $x - \hat{x}_i$ is the difference between $x$ and the i-th training sample
  - I.e. the value at $x$ of the kernel centered on $\hat{x}_i$

**Changing the kernel function:**

* Allows to adjust the properties of the distribution (e.g. smoothness)
* ...By exploiting our prior knowledge

## Kernel Density Estimation

An example with two samples and a Guassian kernel:

In [9]:
x = np.array([0.25, 0.75]).reshape(-1,1) # two sample, univariate
kde = KernelDensity(kernel='gaussian', bandwidth=0.1) # build the estimator
kde.fit(x) # fit the estimator on the data
nab.plot_density_estimator_1D(kde, xr=np.linspace(0, 1, 200))
ymin, ymax = plt.ylim()
plt.vlines(x, ymin, ymax, color='tab:red')
plt.ylim((ymin, ymax)); # ; = suppress output

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Kernel Density Estimation

**How do we tune the bandwidth?**

A general approach consists in:

* Choosing a _validation set_
* _Maximizing its likelihood_ according to the model

More on that later!

**A rule of thumb for the univariate case:**

This is obtain via some assumptions and statistical considerations

$$h = 0.9 \min\left(\hat{\sigma}, \frac{\mathit{IQR}}{1.34}\right) m^{-\frac{1}{5}}$$

* $\mathit{IQR}$ is the inter-quartile range